## 1 典型应用
仅仅到如APK对象，进行简单的内容提取，不需要复杂的分析工作。

In [ ]:
#!/usr/local/python3/bin/python3.6
 
from androguard.core.bytecodes.apk import APK
import sys
 
filename = ""
if len(sys.argv) > 1:
    filename = sys.argv[1]
if not filename.endswith(".apk"):
    print("error: not a apk file")
    sys.exit(1)
 
a = APK(filename)
if a.get_package is not None:
    print("name=" +  a.get_package())
if a.get_app_name is not None:
    print("appName=" +  a.get_app_name())
if a.get_target_sdk_version is not None:
    print("sdkVersion=" + a.get_target_sdk_version())
if a.get_androidversion_code is not None:
    print("versionCode=" + a.get_androidversion_code())
if a.get_androidversion_code is not None:
    print("versionName=" + a.get_androidversion_name())
for m in a.get_permissions():
    print("permission=" + m)

## 2 一个网络教程——作为python的模块使用
> 这是一个新版本的应用教程 https://blog.csdn.net/vitA_gsm/article/details/104791870?utm_medium=distribute.pc_aggpage_search_result.none-task-blog-2~aggregatepage~first_rank_ecpm_v1~rank_v31_ecpm-9-104791870.pc_agg_new_rank&utm_term=androguard+python&spm=1000.2123.3001.4430 
> 
> 这是一个老版本的教程，需要手动引入很多库文件。可以完全用这种方式进行分析。 https://blog.csdn.net/ybdesire/article/details/52280303

In [ ]:
import os
# 引入androguard的路径，根据个人存放的位置而定
# androguard_module_path = os.path.join( os.path.dirname(os.path.abspath(__file__)), 'androguard' )
# androguard_module_path = '/home/ykl/.virtualenvs/pytorch/bin/androguard'

# if not androguard_module_path in sys.path:
#     sys.path.append(androguard_module_path)
    
from androguard.misc import AnalyzeAPK
from androguard.core.androconf import load_api_specific_resource_module

path = "./apk/"
out_path = "./out"
files = []
path_list=os.listdir(path)
path_list.sort()
for name in path_list:
    if os.path.isfile(os.path.join(path, name)):
        files.append(name)
print(files)


['1.apk', '2.apk', '3.apk', '4.apk', '5.apk']


In [ ]:

def main():
    for apkfile in files:
        file_name = os.path.splitext(apkfile)[0]   
        out = AnalyzeAPK(path + '/' + apkfile) 
        dx = out[2]

        #api和权限映射        
        api_perm_filename = os.path.join(out_path, file_name + "_api-perm.txt")
        api_perm_file = open(api_perm_filename, 'w', encoding='utf-8')
        permmap = load_api_specific_resource_module('api_permission_mappings')
        # print(permmap.items())
        results = []
        for meth_analysis in dx.get_methods():
            meth = meth_analysis.get_method()
            name  = meth.get_class_name() + "-" + meth.get_name() + "-" + str(meth.get_descriptor())       
            for k,v in permmap.items():  
                if name  == k:
                    result = str(meth) + ' : ' + str(v)
                    api_perm_file.write(result + '\n')  
        api_perm_file.close()

main()

## 3 一个网络教程——作为命令行脚本启动
> 这是一个使用命令行分析的详细的教程，能够脚本获取自己所需要的内容。 https://blog.csdn.net/qq_40644809/article/details/106814146?utm_medium=distribute.pc_aggpage_search_result.none-task-blog-2~aggregatepage~first_rank_ecpm_v1~rank_v31_ecpm-20-106814146.pc_agg_new_rank&utm_term=androguard+python&spm=1000.2123.3001.4430
> 
> 这是详解a.d.dx.misc.axml.asrc等一系列对象的内置方法的文件。 https://www.jianshu.com/p/079e40800ef4

```
ykl@ykl-OptiPlex-3070:~/gitee/notes/Python/androguard/apk$ androguard analyze 1.apk
Please be patient, this might take a while.
Found the provided file is of type 'APK'
[INFO    ] androguard.analysis: End of creating cross references (XREF)
[INFO    ] androguard.analysis: run time: 0min 00s
Added file to session: SHA256::000e0948176bdec2b6e19d0f03e23f37910676a9b7e7709954614bac79269c36
Loaded APK file...
>>> a
<androguard.core.bytecodes.apk.APK object at 0x7fc1d37d73d0>
>>> d
[<androguard.core.bytecodes.dvm.DalvikVMFormat object at 0x7fc1d225abe0>]
>>> dx
<analysis.Analysis VMs: 1, Classes: 22, Strings: 2>

Androguard version 3.3.5 started
In [1]: a
Out[1]: <androguard.core.bytecodes.apk.APK at 0x7fc1d37d73d0>

In [2]: d
Out[2]: [<androguard.core.bytecodes.dvm.DalvikVMFormat at 0x7fc1d225abe0>]

In [3]: dx
Out[3]: <analysis.Analysis VMs: 1, Classes: 22, Strings: 2>

In [4]: a.get_permissions()
Out[4]: 
['android.permission.EXPAND_STATUS_BAR',
 'android.permission.ACCESS_NETWORK_STATE',
 'android.permission.WRITE_CALENDAR',
 'android.permission.GET_ACCOUNTS',
 'android.permission.BATTERY_STATS',
 'android.permission.INTERNET',
 'android.permission.SEND_SMS',
 'android.permission.INSTALL_PACKAGES',
 'android.permission.WRITE_CONTACTS',
 'android.permission.CONTROL_LOCATION_UPDATES',
 'android.permission.ACCESS_WIFI_STATE',
 'android.permission.READ_LOGS',
 'android.permission.READ_FRAME_BUFFER',
 'android.permission.READ_CONTACTS',
 'android.permission.CAMERA',
 'android.permission.SYSTEM_ALERT_WINDOW',
 'android.permission.STATUS_BAR',
 'com.android.browser.permission.READ_HISTORY_BOOKMARKS',
 'android.permission.CHANGE_CONFIGURATION',
 'android.permission.VIBRATE',
 'android.permission.READ_CALENDAR']
```
